## Resumen General del Código

En este código se consolida la información de los códigos provenientes de la georreferenciación y de la información dispuesta en el R1, en la búsqueda de los casos donde se puede cruzar información a partir de la matrícula inmobiliaria. De esta forma, este código devuelve la base final, consolidando todos estos proceso. Los pasos de ejecución se detallan a continuación:

Los pasos de ejecución son los siguientes:

- Ejecución de las librerías requeridas.
- Lectura de las tablas requeridas para el análisis a realizar.
- Se debe garantizar la existencia de las tablas en las rutas que se especifican en el programa.
- Alistamiento de las variables. Se realizan los cruces, pegues y transformaciones requeridas.
- Procesamiento sobre las matrículas
- Cruce de la base georreferenciada con el R1
- Alistamiento final de la base y validaciones correspondientes.
- Export de resultados

In [ ]:
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
from unidecode import unidecode
from fuzzywuzzy import process

import os
import zipfile
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import Levenshtein
import datetime

In [23]:
df_R1_matricula = pd.read_csv("../../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Consolidado_4_mayo_Matricula.csv", encoding = "latin1")
df_R1_matricula["CODIGO_ORIP"] = df_R1_matricula["CODIGO_ORIP"].astype(str).str.replace(".0", "")

df_R1_matricula["MUNICIPIO"] = df_R1_matricula["MUNICIPIO"].astype(int).apply(round).astype(str).str.zfill(3)
df_R1_matricula["DEPARTAMENTO"] = df_R1_matricula["DEPARTAMENTO"].astype(int).apply(round).astype(str).str.zfill(2)
df_R1_matricula["DIVIPOLA"] = df_R1_matricula["DEPARTAMENTO"] + df_R1_matricula["MUNICIPIO"]
#ofertas_final["Municipio_x"] = ofertas_final["Municipio_x"].astype(int).apply(round).astype(str).str.zfill(3)
#ofertas_final["Departamento_x"] = ofertas_final["Departamento_x"].astype(int).apply(round).astype(str).str.zfill(2)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_2720\3658018204.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_R1_matricula = pd.read_csv("../../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Consolidado_4_mayo_Matricula.csv", encoding = "latin1")


In [24]:
# df_matricula_R1 = pd.read_csv("../input/20230526_R1_Matricula.csv")
# df_matricula_R1_v2 = df_matricula_R1[df_matricula_R1["MATRICULA_INMOBILIARIA"].notnull()][["NPN", "MATRICULA_INMOBILIARIA", "NUMERO_DEL_PREDIO", 'Departamento', 'Municipio', "DEPARTAMENTO", "DIVIPOLA"]]
# df_matricula_R1_v2 = df_matricula_R1_v2.rename(columns={"NPN" : "NPN_R1", "MATRICULA_INMOBILIARIA" : "MATRICULA_INMOBILIARIA_R1", "NUMERO_DEL_PREDIO" : "NUMERO_DEL_PREDIO_R1", 'Departamento' : 'Departamento_R1', 'Municipio' : 'Municipio_R1', "DEPARTAMENTO" : "CODIGO_DEPARTAMENTO_R1", "DIVIPOLA" : "DIVIPOLA_R1"})
# df_matricula_R1_v2.columns
# #df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1_v2"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
# df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
# pattern = r'[^a-zA-Z0-9]'

#df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'] = df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'].astype(str).str.replace(pattern, '', regex = True)

# df_matricula_R1_v3 = df_matricula_R1_v2[df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1'].str.count("-") == 1]
# df_matricula_R1_v3['Matricula'] = df_matricula_R1_v3['MATRICULA_INMOBILIARIA_R1'].str.extract(r'-(\d+)')
# df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_matricula_R1_v3['DIVIPOLA_R1'].astype(int).apply(round).astype(str).str.strip() + "-" + df_matricula_R1_v3['Matricula'].astype(str).str.strip()

In [25]:
divipola = pd.read_excel("../input/CODIGOS_DIVIPOLA.xlsx", "Listado Vigentes")

divipola["Nombre Municipio"] = divipola["Nombre Municipio"].astype(str).str.strip().apply(lambda x: unidecode(x))
divipola["Nombre Departamento"] = divipola["Nombre Departamento"].astype(str).str.strip().apply(lambda x: unidecode(x))

divipola["Código Municipio"] = divipola["Código Municipio"].astype(int).apply(round)
divipola["Código Departamento"] = divipola["Código Departamento"].astype(int).apply(round)

In [26]:
ofertas_final = pd.read_csv("../output/230725_RESULTADOS_GEOREF.csv")
ofertas_final.columns
ofertas_final.drop(['Direccion_y', 'Municipio_y','Departamento_y', 'Código Departamento', 'Código Municipio_y', 'Código Departamento', 'Código Municipio_y', 'Código Municipio_x', 'Nombre Municipio', 'Nombre Departamento'], axis = 1, inplace=True)
ofertas_final.rename({"Direccion_x": "Direccion", "Departamento_x" : "Departamento", "Municipio_x" : "Municipio"}, inplace = True, axis = 1)
ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "DIVIPOLA"] = ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "Municipio"]
ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "DIVIPOLA"] = ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "Municipio"]
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(str).str.replace("\.0+", "", regex = True)
ofertas_final.loc[ofertas_final["DIVIPOLA"].str.contains("\D", regex = True), "DIVIPOLA"] = None
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].fillna(0)
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(int).apply(round).astype(str).str.zfill(5)
ofertas_final["Municipio"] = ofertas_final["DIVIPOLA"]
ofertas_final['Departamento'] = ofertas_final['DIVIPOLA'].apply(lambda x: x[:2])

C:\Users\Administrador\AppData\Local\Temp\ipykernel_2720\2671017565.py:1: DtypeWarning: Columns (5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,32,33,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  ofertas_final = pd.read_csv("../output/230725_RESULTADOS_GEOREF.csv")


In [27]:
divipola["Código Municipio"] = divipola["Código Municipio"].astype(int).apply(round).astype(str).str.zfill(5)
ofertas_final = pd.merge(ofertas_final, divipola[["Código Municipio", "Nombre Municipio","Nombre Departamento"]].drop_duplicates(), left_on = "DIVIPOLA", right_on = "Código Municipio", how = "left")
ofertas_final["Municipio_nombre"] = ofertas_final["Nombre Municipio"]
ofertas_final["Departamento_nombre"] = ofertas_final["Nombre Departamento"]
ofertas_final.drop(["Código Municipio", "Nombre Municipio","Nombre Departamento"], axis = 1, inplace = True)

In [28]:
tabla_codigos_orip = pd.read_excel("../input/tablaORIP.xlsx", sheet_name = "Hoja1")

#tabla_codigos_orip["CÓDIGO ORIP"] = tabla_codigos_orip["CÓDIGO ORIP"].str.strip().astype(str)
tabla_codigos_orip["DIVIPOLA"] = tabla_codigos_orip["DIVIPOLA"].fillna(0)
tabla_codigos_orip["DIVIPOLA"] = tabla_codigos_orip["DIVIPOLA"].astype(str).str.replace(".0", "").str.zfill(5)
tabla_codigos_orip[tabla_codigos_orip["DIVIPOLA"].astype(str).str.contains("25168")]

,CÓDIGO ORIP,DEPARTAMENTO,COBERTURA,DIVIPOLA,NOMBRE OFICINA DE REGISTRO,NOMBRE ORIP,FECHA DE CREACIÓN,ACTO ADMINISTRATIVO DE CREACIÓN,JEFE DE OFICINA,CORREO JEFE,CONTACTO
603,162,CUNDINAMARCA,CHAGUANÍ,25168,GUADUAS,NaN,NaN,NaN,DAVID RAMIREZ GUERRA,david.ramirez@supernotariado.gov.co,(1) 8417430 (1)8466266


In [29]:
codigos_orip_divipola = tabla_codigos_orip[["DIVIPOLA", "CÓDIGO ORIP"]].drop_duplicates("DIVIPOLA")
df_R1_matricula["DIVIPOLA"] = df_R1_matricula["DIVIPOLA"].astype(str)
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(str)
codigos_orip_divipola["DIVIPOLA"] = codigos_orip_divipola["DIVIPOLA"].astype(str)
df_R1_matricula = pd.merge(df_R1_matricula, codigos_orip_divipola, how = "left", on = "DIVIPOLA")
ofertas_final = pd.merge(ofertas_final, codigos_orip_divipola, how = "left", on = "DIVIPOLA")

In [30]:
df_matriculas = pd.DataFrame({"Matricula_Inmobiliaria" : ofertas_final["Matricula_Inmobiliaria"].str.strip().str.extractall(r'(\d+)')[0].reset_index(level=1, drop=True)})
df_matriculas.tail()

df_merge = pd.merge(df_matriculas, ofertas_final[["CÓDIGO ORIP"]], how = "left", left_index = True, right_index = True)

df_merge["Matricula_Divipola"] = df_merge["CÓDIGO ORIP"].astype(str) + "-" + df_merge["Matricula_Inmobiliaria"].astype(str)
#ofertas_final[["DIVIPOLA"]]

In [31]:
df_R1_matricula["MATRICULA"].fillna(0, inplace=True)
df_R1_matricula = df_R1_matricula[df_R1_matricula["NUMERO_DE_ORDEN"].isin([1, "1"])]
df_R1_matricula["MATRICULA"] = df_R1_matricula["MATRICULA"].astype(str).str.replace(".0", "").str.strip()
df_R1_matricula["NPN_R1"] = df_R1_matricula["DEPARTAMENTO"] + df_R1_matricula["MUNICIPIO"] + df_R1_matricula["NUMERO_DEL_PREDIO"]
#df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_R1_matricula['CÓDIGO ORIP'].astype(str).str.strip() + "-" + df_R1_matricula['MATRICULA'].astype(str).str.strip()
#df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'].str.strip().str.replace(" ", "").astype(str)
df_merge['Matricula_Divipola'] = df_merge['Matricula_Divipola'].str.strip().str.replace(" ", "").astype(str)
df_merge.reset_index(inplace= True)
df_merge_2 = pd.merge(df_merge, df_R1_matricula[["CÓDIGO ORIP", "MATRICULA", "DEPARTAMENTO", "MUNICIPIO", "NUMERO_DEL_PREDIO", "NPN_R1"]].drop_duplicates(), how = "inner", left_on = ["CÓDIGO ORIP", "Matricula_Inmobiliaria"], right_on = ["CÓDIGO ORIP", "MATRICULA"])
df_merge_2.drop_duplicates("index",inplace= True)
#df_merge_2.set_index("index", inplace= True)
df_merge_2.shape
	

(33459, 9)

In [32]:
ORIP_UNIQUE_CODES = df_merge[df_merge["CÓDIGO ORIP"].isin(df_R1_matricula["CÓDIGO ORIP"])]["CÓDIGO ORIP"].unique()

df_aprox_matriculas_final = []
for orip in ORIP_UNIQUE_CODES:
    orip = ORIP_UNIQUE_CODES[0]
    df_avaluos = df_merge[df_merge["CÓDIGO ORIP"].astype(str) == str(orip)].copy()
    df_matricula_orip = df_R1_matricula[df_R1_matricula["CÓDIGO ORIP"].astype(str) == str(orip)].copy()

    df_avaluos = df_avaluos[~df_avaluos["Matricula_Inmobiliaria"].str.contains("\D", regex = True)]
    df_matricula_orip = df_matricula_orip[["CÓDIGO ORIP", "MATRICULA", "DEPARTAMENTO", "MUNICIPIO", "NUMERO_DEL_PREDIO", "NPN_R1"]].drop_duplicates()
    df_matricula_orip = df_matricula_orip[~df_matricula_orip["MATRICULA"].str.contains("\D", regex = True)]

    #df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype(int)

    #df_avaluos.sort_values("Matricula_Inmobiliaria")
    #df_avaluos
    #df_matricula_orip[["MATRICULA"]]
    df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype("Int64")
    df_matricula_orip["MATRICULA"] = df_matricula_orip["MATRICULA"].astype("Int64")
    df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype("int")
    df_matricula_orip["MATRICULA"] = df_matricula_orip["MATRICULA"].astype("int")
    df_avaluos.sort_values("Matricula_Inmobiliaria", inplace=True)
    df_matricula_orip.sort_values("MATRICULA", inplace=True)

    df_avaluos_matricula_aprox = pd.merge_asof(df_avaluos, df_matricula_orip[["MATRICULA","NPN_R1"]], left_on = "Matricula_Inmobiliaria", right_on = "MATRICULA")

    # Function to calculate Levenshtein distance
    def calculate_distance(row):
        return Levenshtein.distance(row['Matricula_Inmobiliaria'], row['MATRICULA'])

    df_avaluos_matricula_aprox["Matricula_Inmobiliaria"] = df_avaluos_matricula_aprox["Matricula_Inmobiliaria"].astype(str)
    df_avaluos_matricula_aprox["MATRICULA"] = df_avaluos_matricula_aprox["MATRICULA"].astype(str)
    # Apply the function to each row and store the results in a new column
    df_avaluos_matricula_aprox['distance'] = df_avaluos_matricula_aprox.apply(calculate_distance, axis=1)
    df_avaluos_matricula_aprox['Longitud_matricula'] = df_avaluos_matricula_aprox["Matricula_Inmobiliaria"].apply(len)
    df_aprox_matriculas_final.append(df_avaluos_matricula_aprox)
    #df_avaluos_matricula_aprox.tail()


In [33]:
df_aprox_matriculas_final_2 = pd.concat(df_aprox_matriculas_final, axis = 0)
df_aprox_matriculas_final_3 = df_aprox_matriculas_final_2[(df_aprox_matriculas_final_2["distance"] >= 1) & (df_aprox_matriculas_final_2["distance"] < 2) & (df_aprox_matriculas_final_2["Longitud_matricula"] > 4)]
df_aprox_matriculas_final_3["MATRICULA_APROXIMADA"] = df_aprox_matriculas_final_3["CÓDIGO ORIP"].astype(str) + "-" + df_aprox_matriculas_final_3["MATRICULA"].astype(str)
df_aprox_matriculas_final_4 = df_aprox_matriculas_final_3[["index", "MATRICULA_APROXIMADA", "NPN_R1"]]
df_aprox_matriculas_final_4.rename({"NPN_R1": "NPN_R1_APROX"}, axis = 1, inplace=True)
df_aprox_matriculas_final_4.set_index("index", inplace=True)
df_aprox_matriculas_final_4.head()


C:\Users\Administrador\AppData\Local\Temp\ipykernel_2720\4173479554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_3["MATRICULA_APROXIMADA"] = df_aprox_matriculas_final_3["CÓDIGO ORIP"].astype(str) + "-" + df_aprox_matriculas_final_3["MATRICULA"].astype(str)
C:\Users\Administrador\AppData\Local\Temp\ipykernel_2720\4173479554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_4.rename({"NPN_R1": "NPN_R1_APROX"}, axis = 1, inplace=True)


,MATRICULA_APROXIMADA,NPN_R1_APROX
index,,
12920,200-10126,413570000000000130035000000000
483134,200-13461,410200101000000230015000000000
7061,200-13550,410200400000000080001000000000
24551,200-14431,413570000000000100044000000000
7193,200-15522,411320000000000330032000000000


In [34]:
df_aprox_matriculas_final_3.head()

,index,Matricula_Inmobiliaria,CÓDIGO ORIP,Matricula_Divipola,MATRICULA,NPN_R1,distance,Longitud_matricula,MATRICULA_APROXIMADA
511,12920,10127,200,200-10127,10126,413570000000000130035000000000,1,5,200-10126
522,483134,13466,200,200-00013466,13461,410200101000000230015000000000,1,5,200-13461
523,7061,13555,200,200-13555,13550,410200400000000080001000000000,1,5,200-13550
526,24551,14432,200,200-14432,14431,413570000000000100044000000000,1,5,200-14431
531,7193,15529,200,200-15529,15522,411320000000000330032000000000,1,5,200-15522


In [35]:
df_merge_2.set_index("index", inplace = True)
df_merge_2["FUENTE"] = "R1_MATRICULA"

In [36]:
df_aprox_matriculas_final_4.reset_index(inplace=True)
df_aprox_matriculas_final_4.drop_duplicates(inplace=True)
df_aprox_matriculas_final_4.set_index("index", inplace=True)


C:\Users\Administrador\AppData\Local\Temp\ipykernel_2720\2358486246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_4.drop_duplicates(inplace=True)


In [37]:
ofertas_final_v1 = pd.merge(ofertas_final, df_merge_2[["NPN_R1", "FUENTE", "Matricula_Divipola"]], how = "left", left_index = True, right_index = True)
ofertas_final_v2 = pd.merge(ofertas_final_v1,  df_aprox_matriculas_final_4, how = "left", left_index = True, right_index = True)
ofertas_final_v2.shape
#pd.merge

(503568, 62)

In [38]:
quantiles = list(np.arange(0, 0.95, 0.05))#[0.05, 0.1, 0.2, 0.25, 0.50, 0.75, 0.90, 0.95, 0.97]
ofertas_final_v2[(ofertas_final_v2["ZONA"] == "RURAL_APROX") & (ofertas_final_v2["FUENTE"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()



,index,Distancia_final
0,0.000000,0.000003
1,0.050000,0.000096
2,0.100000,0.002973
3,0.150000,0.004204
4,0.200000,0.006006
5,0.250000,0.006340
6,0.300000,0.006340
7,0.350000,0.006803
8,0.400000,0.007404
9,0.450000,0.007621


In [39]:
pd.options.display.float_format = '{:.6f}'.format
ofertas_final_v2[(ofertas_final_v2["ZONA"] == "URBANA_APROX") & (ofertas_final_v2["FUENTE"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()

,index,Distancia_final
0,0.000000,0.000000
1,0.050000,0.000002
2,0.100000,0.000005
3,0.150000,0.000008
4,0.200000,0.000008
5,0.250000,0.000013
6,0.300000,0.000019
7,0.350000,0.000026
8,0.400000,0.000037
9,0.450000,0.000045


In [40]:
ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "CODIGO_PREDIAL"] = ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "NPN_R1"]
#ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "Fuente"] = "OIC"
ofertas_final_v2.loc[(ofertas_final_v2["NPN_R1"].isnull()) & (ofertas_final_v2["CODIGO_PREDIAL"].notnull()), "FUENTE"] = "Subdireccion"

ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.000092) & (ofertas_final_v2["ZONA"] == "URBANA_APROX"), "TIPO_GEORREF"] = "APROX_OK"
ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.008925) & (ofertas_final_v2["ZONA"] == "RURAL_APPROX"), "TIPO_GEORREF"] = "APROX_OK"

# ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.000045) & (ofertas_final_v2["ZONA"] == "URBANA_APROX"), "TIPO_GEORREF"] = "APROX_OK"
# ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.007404) & (ofertas_final_v2["ZONA"] == "RURAL_APPROX"), "TIPO_GEORREF"] = "APROX_OK"


In [41]:
# Get today's date
today = datetime.date.today()
# Format today's date as YYMMDD
formatted_date = today.strftime("%y%m%d")

ofertas_final_v2["SUBSTRING_67"] = ofertas_final_v2["CODIGO_PREDIAL"].str[5:7]
ofertas_final_v2["SUBSTRING_22"] = ofertas_final_v2["CODIGO_PREDIAL"].str[21:22]

# Define the conditions and corresponding values
conditions = [
    ofertas_final_v2['SUBSTRING_67'] == "00",
    ofertas_final_v2['SUBSTRING_67'] == "01",
    (~ofertas_final_v2['SUBSTRING_67'].isin(["01", "00"])) & (ofertas_final_v2['SUBSTRING_67'].notnull())
]
choices = ['RURAL', 'URBANO', 'CENTRO_P']

# Apply the case when statement using numpy.select()
ofertas_final_v2['ZONA'] = np.select(conditions, choices, default='SIN_ASIGNAR')

conditions_tipo_predio = [
    ofertas_final_v2['SUBSTRING_22'] == "9",
    ofertas_final_v2['SUBSTRING_22'] == "8",
    ofertas_final_v2['SUBSTRING_22'] == "7",
    ofertas_final_v2['SUBSTRING_22'] == "5",
    ofertas_final_v2['SUBSTRING_22'] == "4",
    ofertas_final_v2['SUBSTRING_22'] == "3",
    ofertas_final_v2['SUBSTRING_22'] == "0"    
]
choices_tipo_predio = ["PH", "CONDOMINIO", "CEMENTERIOS", "MEJORA", "VIAS", "USO_PUBLICO", "NPH"]


ofertas_final_v2['CLASE_PREDIO'] = np.select(conditions_tipo_predio, choices_tipo_predio, default='SIN_ASIGNAR')

ofertas_final_v2.loc[ofertas_final_v2["FUENTE"] != "R1_MATRICULA", 'CLASE_PREDIO'] = "SIN ASIGNAR"

ofertas_final_v2["DIVIPOLA_EN_R1"] = "NO"
ofertas_final_v2.loc[ofertas_final_v2["DIVIPOLA"].isin(df_R1_matricula["DIVIPOLA"].drop_duplicates().to_list()), "DIVIPOLA_EN_R1"] = "SI"

ofertas_final_v2.loc[(ofertas_final_v2["DIVIPOLA_EN_R1"] == "NO") & (ofertas_final_v2["FUENTE"] == "R1_MATRICULA"), "FUENTE"] = "MUNICIPIO_SIN_R1"


ofertas_final_v2.reset_index(inplace = True)

Logical_validation_mun = ofertas_final_v2[(ofertas_final_v2["FUENTE"] == "Subdireccion") & (ofertas_final_v2["TIPO_GEORREF"].isin(["EXACTA", "APROX_OK"]))]["CODIGO_PREDIAL"].str[0:5] != ofertas_final_v2[(ofertas_final_v2["FUENTE"] == "Subdireccion") & (ofertas_final_v2["TIPO_GEORREF"].isin(["EXACTA", "APROX_OK"]))]["DIVIPOLA"]
ofertas_final_v2.loc[ofertas_final_v2["index"].isin(Logical_validation_mun[Logical_validation_mun == True].index), "TIPO_GEORREF"] = "APROX"

ofertas_final_v2.set_index("index")

ofertas_final_v2["ENCONTRADO"] = "NO"
ofertas_final_v2.loc[(ofertas_final_v2["FUENTE"] == "R1_MATRICULA") | ((ofertas_final_v2["FUENTE"] == "Subdireccion") & (ofertas_final_v2["TIPO_GEORREF"].isin(["EXACTA", "APROX_OK"]))), "ENCONTRADO"] = "SI"
ofertas_final_v2.drop(['Unnamed: 0', 'Unnamed: 28', 'Direccion1', 'address',  'Consecutivo',  "SUBSTRING_22", 'SUBSTRING_67', "CODIGO_ANT_RURAL_APROX", "CODIGO_ANT_URBANO", "CODIGO_ANT_RURAL","CODIGO_ANT_URBANO_APROX"], axis = 1, inplace = True)

ofertas_final_v2.to_csv("../output/" + formatted_date + "_BASE_PROCESADA_AVALUOS_IGAC.csv")
ofertas_final_v2.to_excel("../output/" + formatted_date + "_BASE_PROCESADA_AVALUOS_IGAC.xlsx")

In [42]:
ofertas_final_v2["ENCONTRADO"].value_counts(dropna = False)

ENCONTRADO
NO    452432
SI     51136
Name: count, dtype: int64

In [44]:
ofertas_final_v2[["DIVIPOLA_EN_R1","ENCONTRADO"]].value_counts(dropna = False)

DIVIPOLA_EN_R1  ENCONTRADO
NO              NO            408287
SI              SI             51136
                NO             44145
Name: count, dtype: int64

In [69]:
ofertas_final_v2.drop_duplicates().shape

(503568, 57)

In [53]:
CHECK = ofertas_final_v2[ofertas_final_v2["DIVIPOLA_EN_R1"] == "NO"]["DIVIPOLA"].value_counts(dropna = False).to_frame()

In [66]:
#df_R1_matricula[df_R1_matricula["DIVIPOLA"] == "76130"]
#CHECK.iloc[0:10]["count"].sum()
ofertas_final_v2[["DIVIPOLA_EN_R1","ENCONTRADO", "FUENTE"]].value_counts(dropna = False).to_frame().to_excel("../output/RESUMEN_EXCLUSIONES.xlsx")